In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error
import random
from tensorflow.keras.models import load_model
import scipy
import matplotlib.pyplot as plt
np.random.seed(0)

2022-12-09 21:34:09.397547: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-09 21:34:09.478282: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 21:34:09.478300: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-12-09 21:34:10.050315: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2022-

In [2]:
url = 'https://raw.githubusercontent.com/igomezv/nnogada/main/data/jla.csv'
df=pd.read_csv(url)
N = len(df.values)
randomize = np.random.permutation(N)
data = df.values[randomize]
N = len(df.values)
z = data[:,0] 
y = data[:,1:3] ### coge el resto de variables a predecir 
y[:,1] = y[:,1]**2+data[:,2]
np.shape(y)
df

,zcmb,mb,dmb,errors
0,0.503084,23.001698,0.088031,0.120219
1,0.580724,23.573937,0.090132,0.121435
2,0.494795,22.960139,0.088110,0.089552
3,0.345928,22.398137,0.087263,0.119729
4,0.677662,24.078115,0.098356,0.100088
...,...,...,...,...
735,0.027064,16.504006,0.141685,0.185894
736,0.025468,15.797848,0.143429,0.193666
737,0.023810,15.895501,0.144315,0.184957
738,0.023867,16.068268,0.144350,0.175960


In [3]:
dmag =df["dmb"]

df2=df['errors']+df['dmb']**2

In [4]:
scalerz = StandardScaler()
scalerz.fit(z.reshape(-1,1))
z = scalerz.transform(z.reshape(-1,1))

In [5]:
### Modificar para incluir phanteon como test
split = 0.75
ntrain = int(split * len(z))
indx = [ntrain]
X_train, X_test = np.split(z, indx)
Y_train, Y_test = np.split(y, indx)
np.shape(X_train), np.shape(X_test), Y_train.shape, Y_test.shape

((555, 1), (185, 1), (555, 2), (185, 2))

In [6]:
model1 = load_model("models/model_jla_grid.h5")
model_gena = load_model("models/modela_jla_genetic.h5")
model_genb = load_model("models/modelb_jla_genetic.h5")
model_genc = load_model("models/modelc_jla_genetic.h5")

2022-12-09 21:34:11.232921: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-12-09 21:34:11.233107: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-12-09 21:34:11.233174: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-12-09 21:34:11.233234: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-12-09 21:34:11.233294: W tensorflow/c

In [7]:
# ranz = np.random.uniform(0, 2, size=10000)
ranz = np.linspace(np.min(df['zcmb']), np.max(df['zcmb']), 1000)
r = scalerz.transform(ranz.reshape(-1, 1))
pred_random_gena = model_gena.predict(r)
pred_random_genb = model_genb.predict(r)
pred_random_genc = model_genc.predict(r)
rr = scalerz.inverse_transform(r.reshape(-1,1))
pred_random_grid = model1.predict(r)

32/32 [==============================] - 0s 731us/step


In [8]:
y1 = model1.predict(X_test)
ya = model_gena.predict(X_test)
yb = model_genb.predict(X_test)
yc = model_genc.predict(X_test)

6/6 [==============================] - 0s 690us/step


In [10]:
r1 = r2_score(y1, Y_test)
r2 = r2_score(ya, Y_test)
r3 = r2_score(yb, Y_test)
r4 = r2_score(yc, Y_test)
r1, r2, r3, r4

(0.36396486222373603,
 0.6927487706262881,
 0.7264085317799915,
 0.6162283748464275)

In [11]:
mae1 = mean_absolute_error(y1, Y_test)
mae2 = mean_absolute_error(ya, Y_test)
mae3 = mean_absolute_error(yb, Y_test)
mae4 = mean_absolute_error(yc, Y_test)

mae1, mae2, mae3, mae4

(0.11921155338078253,
 0.10146906916845636,
 0.10715060130821545,
 0.10084156245679599)